<a href="https://colab.research.google.com/github/khushi6120-cloud/Tech-assesment/blob/main/speech_label1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install speechrecognition
import speech_recognition as sr

     |████████████████████████████████| 32.8 MB 49.4 MB/s 


## GPU Training ===

## Loading dataset

In [ ]:
dataset_train = pd.read_csv('train_data.csv')

# In complete model replace X_train with a speech to text model's output so that model would work on that

X_train = dataset_train.iloc[:, 1:2].values # 1:2 writter as python ignors the upper bound
# values -- it is done so to make a numpy array rather than a simple vector
paths = dataset_train.iloc[:,0:1]

In [ ]:
r = sr.Recognizer()

In [ ]:
print(paths.iloc[0,0:1])

path    wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...
Name: 0, dtype: object


In [ ]:
print(paths)

                                                    path
0      wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...
1      wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...
2      wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...
3      wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...
4      wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...
...                                                  ...
11561  wavs/speakers/BvyakyrDmQfWEABb/a59b2910-4609-1...
11562  wavs/speakers/W4XOzzNEbrtZz4dW/aa186590-44d1-1...
11563  wavs/speakers/xwpvGaaWl5c3G5N3/6e189850-45ba-1...
11564  wavs/speakers/zaEBPeMY4NUbDnZy/3fd3a020-44d4-1...
11565  wavs/speakers/zaEBPeMY4NUbDnZy/3c3d7250-44da-1...

[11566 rows x 1 columns]


In [ ]:
for i in range(0,11566):
  harvad = sr.AudioFile('/' + paths.iloc[i,0:1])
  with harvad as source:
    audio = r.record(source)
  val = r.recognize_google(audio)
  X_train.iloc[i,0:1] = val
  print(val, end='\n')


In [ ]:
print(len(X_train))
print(len(paths))

11566
11566


## Data preprocessing

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 11566):
  review = re.sub('[^a-zA-Z]', ' ', dataset_train['transcription'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(len(corpus))

11566


## Initializing & traning model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y1 = dataset_train.iloc[:, 2].values
y2 = dataset_train.iloc[:, 3].values
y3 = dataset_train.iloc[:, 4].values

In [ ]:
print(len(y1))

11566


In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y1, test_size = 0.20, random_state = 0)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size = 0.20, random_state = 0)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y3, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier1 = GaussianNB()
classifier2 = GaussianNB()
classifier3 = GaussianNB()
classifier1.fit(X_train1, y_train1)
classifier2.fit(X_train2, y_train2)
classifier3.fit(X_train3, y_train3)

GaussianNB()

## prediction

In [ ]:
y_pred1 = classifier1.predict(X_test1)
y_pred2 = classifier1.predict(X_test2)
y_pred3 = classifier1.predict(X_test3)
print(np.concatenate((y_pred1.reshape(len(y_pred1),1), y_test1.reshape(len(y_test1),1)),1))
print(np.concatenate((y_pred2.reshape(len(y_pred2),1), y_test2.reshape(len(y_test2),1)),1))
print(np.concatenate((y_pred3.reshape(len(y_pred3),1), y_test3.reshape(len(y_test3),1)),1))

[['increase' 'increase']
 ['activate' 'activate']
 ['increase' 'increase']
 ...
 ['increase' 'increase']
 ['decrease' 'decrease']
 ['decrease' 'increase']]
[['increase' 'volume']
 ['activate' 'lights']
 ['increase' 'volume']
 ...
 ['increase' 'volume']
 ['decrease' 'heat']
 ['decrease' 'heat']]
[['increase' 'none']
 ['activate' 'washroom']
 ['increase' 'none']
 ...
 ['increase' 'none']
 ['decrease' 'none']
 ['decrease' 'bedroom']]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
cm1 = confusion_matrix(y_test1, y_pred1)
print(cm1)
accuracy_score(y_test1, y_pred1)


[[131   0   0 244   0   0]
 [  0 209   0   0   0   0]
 [  0   0 271   0   0   0]
 [  0   0   0 342   0   0]
 [  0   0   0   6 534  10]
 [  0   0   0   7 264 296]]


0.7705272255834054

In [ ]:
f1_score(y_test1, y_pred1, average=None)

array([0.51778656, 1.        , 1.        , 0.72688629, 0.79228487,
       0.67812142])

In [ ]:
import pickle
filename = 'khushi_model.sac'
pickle.dump(classifier1, open(filename, 'wb'))

## Rought setting 

In [ ]:
Y_train_lb1 = dataset_train.iloc[:,2] # action label
Y_train_lb2 = dataset_train.iloc[:,3] # Object label
Y_train_lb3 = dataset_train.iloc[:,4] # Location label

In [ ]:
labels_lb1 = set()
labels_lb2 = set()
labels_lb3 = set()
for label in Y_train_lb1:
        if label not in labels_lb1:
            labels_lb1.add(label)
for label in Y_train_lb2:
        if label not in labels_lb2:
            labels_lb2.add(label)
for label in Y_train_lb3:
        if label not in labels_lb3:
            labels_lb3.add(label)


In [ ]:
print(labels_lb1)
print(len(labels_lb1))
print(len(labels_lb2))
print(len(labels_lb3))

{'decrease', 'bring', 'increase', 'activate', 'deactivate', 'change language'}
6
14
4
